In [22]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [23]:
lib.utils.set_seed()
downscale = 14
train_x, train_y = lib.datasets.to_plain_odd(*lib.datasets.get_mnist_test(), downscale=downscale)
network = lib.livenet.LiveNet(784 // (downscale * downscale), 7, 2)
network.context.alpha_l1 = 0.01

In [24]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=100)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.001
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=100)


LiveNet


In [25]:
network.context.alpha_l1 = 0.1
trainer.step(500)


NameError: name 'bu' is not defined

In [5]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.4926

In [28]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [15]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.6f}")

D0 -0.014559
D1 0.014559
N4 -0.004461
N4->D0 0.000630


In [19]:
network._parameters

OrderedDict([('D0',
              Parameter containing:
              tensor(-0.0146, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(0.0146, requires_grad=True)),
             ('N4',
              Parameter containing:
              tensor(-0.0045, requires_grad=True))])

In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

